In [1]:
import altair as alt
from altair.expr import datum, if_
import datetime
import json
import numpy as np
import pandas as pd
import psycopg2
pd.set_option("display.max_columns", 25)
pd.set_option("display.max_rows", 150)
pd.options.display.float_format = "{:,.2f}".format

Connect to the database.

In [2]:
with open("config.json") as f:
    conf = json.load(f)
conn = psycopg2.connect(
    dbname=conf["database"],
    user=conf["user"],
    host=conf["host"],
    password=conf["password"]
)
conn.autocommit = True

## Which families and corporations gave at least $50,000 to conservative and GOP-aligned groups between Nov. 2 and Dec. 31, 2017?

Return contributions by donor and committee.

In [3]:
post_bill_donors_committees_17 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_donors_committees_17 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2017-11-02'
     AND date <= '2017-12-31'
     AND CYCLE = '2018'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2018'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE post_bill_donors_committees_17 TO redash_default;


SELECT *
FROM post_bill_donors_committees_17;""", con=conn)
post_bill_donors_committees_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515 entries, 0 to 514
Data columns (total 6 columns):
match_id         515 non-null object
organizations    515 non-null object
contributors     515 non-null object
total            515 non-null float64
cmte_id          515 non-null object
committee        515 non-null object
dtypes: float64(1), object(5)
memory usage: 24.2+ KB


Group by donor.

In [4]:
post_bill_donors_17 = post_bill_donors_committees_17.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
post_bill_donors_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 158 entries, 0 to 157
Data columns (total 4 columns):
match_id         158 non-null object
organizations    158 non-null object
contributors     158 non-null object
total            158 non-null float64
dtypes: float64(1), object(3)
memory usage: 5.0+ KB


## How much did they give?

In [5]:
post_bill_donors_17["total"].sum()

35639563.0

## Had these megadonors given previously in 2017?

Return contributions by donor and committee.

In [6]:
pre_bill_donors_committees_17 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_donors_committees_17 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2017-01-01'
     AND date < '2017-11-02'
     AND CYCLE = '2018'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2018'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE pre_bill_donors_committees_17 TO redash_default;


SELECT *
FROM pre_bill_donors_committees_17;""", con=conn)
pre_bill_donors_committees_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 861 entries, 0 to 860
Data columns (total 6 columns):
match_id         861 non-null object
organizations    861 non-null object
contributors     861 non-null object
total            861 non-null float64
cmte_id          861 non-null object
committee        861 non-null object
dtypes: float64(1), object(5)
memory usage: 40.4+ KB


Group by donor.

In [7]:
pre_bill_donors_17 = pre_bill_donors_committees_17.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115 entries, 0 to 114
Data columns (total 4 columns):
match_id         115 non-null object
organizations    115 non-null object
contributors     115 non-null object
total            115 non-null float64
dtypes: float64(1), object(3)
memory usage: 3.7+ KB


## How much did they give?

In [8]:
pre_bill_donors_17["total"].sum()

58975282.0

## How much did these donors give in the period leading up to the tax bill's introduction and in the period after its introduction?

In [9]:
donors_17 = post_bill_donors_17.merge(pre_bill_donors_17, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_17.drop(["organizations_pre_bill", "contributors_pre_bill"], axis=1, inplace=True)
donors_17.rename(columns={"organizations_post_bill": "organizations", "contributors_post_bill": "contributors"}, inplace=True)
donors_17["total_pre_bill"].fillna(0, inplace=True)
donors_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 5 columns):
match_id           158 non-null object
organizations      158 non-null object
contributors       158 non-null object
total_post_bill    158 non-null float64
total_pre_bill     158 non-null float64
dtypes: float64(2), object(3)
memory usage: 7.4+ KB


In [10]:
donors_17["pct_post_bill"] = donors_17["total_post_bill"] / (donors_17["total_pre_bill"] + donors_17["total_post_bill"])
donors_17["pct_pre_bill"] = donors_17["total_pre_bill"] / (donors_17["total_pre_bill"] + donors_17["total_post_bill"])
donors_17["change"] = donors_17["total_post_bill"] - donors_17["total_pre_bill"]
donors_17["pct_change"] = (donors_17["total_post_bill"] - donors_17["total_pre_bill"]) / donors_17["total_pre_bill"].abs()
donors_17.sort_values("total_post_bill", ascending=False).head()

,match_id,organizations,contributors,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
52,U0000003690,"[U-Line Corp, Uline Inc]","[UIHLEIN, ELIZABETH, UIHLEIN, ELIZABETH MRS, U...","4,557,300.00","12,051,400.00",0.27,0.73,"-7,494,100.00",-0.62
2,American Action Network,[American Action Network],[AMERICAN ACTION NETWORK],"3,747,520.00","9,142,525.00",0.29,0.71,"-5,395,005.00",-0.59
60,U0000004054,"[Ghpalmer Assoc, GH Palmer Assoc]","[PALMER, GEOFF, PALMER, GEOFFREY H]","1,955,200.00","546,272.00",0.78,0.22,"1,408,928.00",2.58
10,Hillwood Development,[Hillwood Development],[HILLWOOD DEVELOPMENT COMPANY LLC],"1,500,000.00","500,000.00",0.75,0.25,"1,000,000.00",2.00
67,U0000004552,"[Cinemark Holdings, Cinemark USA]","[MITCHELL, LEE, MITCHELL, LEE ROY, MITCHELL, T...","1,007,400.00","131,299.00",0.88,0.12,"876,101.00",6.67


## Had these megadonors given between Nov. 2 and Dec. 31, 2015?

Return contributions by donor and committee.

In [11]:
post_bill_donors_committees_15 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_donors_committees_15 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2015-11-02'
     AND date <= '2015-12-31'
     AND CYCLE = '2016'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2016'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE post_bill_donors_committees_15 TO redash_default;


SELECT *
FROM post_bill_donors_committees_15;""", con=conn)
post_bill_donors_committees_15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328 entries, 0 to 327
Data columns (total 6 columns):
match_id         328 non-null object
organizations    328 non-null object
contributors     328 non-null object
total            328 non-null float64
cmte_id          328 non-null object
committee        328 non-null object
dtypes: float64(1), object(5)
memory usage: 15.5+ KB


Group by donor.

In [12]:
post_bill_donors_15 = post_bill_donors_committees_15.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
post_bill_donors_15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80 entries, 0 to 79
Data columns (total 4 columns):
match_id         80 non-null object
organizations    80 non-null object
contributors     80 non-null object
total            80 non-null float64
dtypes: float64(1), object(3)
memory usage: 2.6+ KB


## How much did they give?

In [13]:
post_bill_donors_15["total"].sum()

1647560.0

## Had these megadonors given previously in 2015?

Return contributions by donor and committee.

In [14]:
pre_bill_donors_committees_15 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_donors_committees_15 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2015-01-01'
     AND date < '2015-11-02'
     AND CYCLE = '2016'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2016'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE pre_bill_donors_committees_15 TO redash_default;


SELECT *
FROM pre_bill_donors_committees_15;""", con=conn)
pre_bill_donors_committees_15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1063 entries, 0 to 1062
Data columns (total 6 columns):
match_id         1063 non-null object
organizations    1063 non-null object
contributors     1063 non-null object
total            1063 non-null float64
cmte_id          1063 non-null object
committee        1063 non-null object
dtypes: float64(1), object(5)
memory usage: 49.9+ KB


Group by donor.

In [15]:
pre_bill_donors_15 = pre_bill_donors_committees_15.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 4 columns):
match_id         109 non-null object
organizations    109 non-null object
contributors     109 non-null object
total            109 non-null float64
dtypes: float64(1), object(3)
memory usage: 3.5+ KB


## How much did they give?

In [16]:
pre_bill_donors_15["total"].sum()

77617350.0

## How much did these donors give in each period?

In [17]:
donors_15 = post_bill_donors_15.merge(pre_bill_donors_15, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_15["total_pre_bill"].fillna(0, inplace=True)
donors_15["total_post_bill"].fillna(0, inplace=True)
donors_15.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 111 entries, 0 to 110
Data columns (total 7 columns):
match_id                   111 non-null object
organizations_post_bill    80 non-null object
contributors_post_bill     80 non-null object
total_post_bill            111 non-null float64
organizations_pre_bill     109 non-null object
contributors_pre_bill      109 non-null object
total_pre_bill             111 non-null float64
dtypes: float64(2), object(5)
memory usage: 6.9+ KB


In [18]:
donors_15["pct_post_bill"] = donors_15["total_post_bill"] / (donors_15["total_pre_bill"] + donors_15["total_post_bill"])
donors_15["pct_pre_bill"] = donors_15["total_pre_bill"] / (donors_15["total_pre_bill"] + donors_15["total_post_bill"])
donors_15["change"] = donors_15["total_post_bill"] - donors_15["total_pre_bill"]
donors_15["pct_change"] = (donors_15["total_post_bill"] - donors_15["total_pre_bill"]) / donors_15["total_pre_bill"].abs()
donors_15 = donors_15[["match_id", "organizations_post_bill", "contributors_post_bill",
                       "organizations_pre_bill", "contributors_pre_bill", "total_post_bill",
                       "total_pre_bill", "pct_post_bill", "pct_pre_bill", "change", "pct_change"]]
donors_15.head()

,match_id,organizations_post_bill,contributors_post_bill,organizations_pre_bill,contributors_pre_bill,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
0,American Action Network,[American Action Network],[AMERICAN ACTION NETWORK],[American Action Network],[AMERICAN ACTION NETWORK],"102,825.00","74,649.00",0.58,0.42,"28,176.00",0.38
1,Chickasaw Nation,[Chickasaw Nation],"[CHICKASAW NATION, CHICKASAW NATION, THE, NATI...",[Chickasaw Nation],"[CHICKASAW NATION, CHICKASAW NATION, THE, NATI...","44,300.00","226,000.00",0.16,0.84,"-181,700.00",-0.80
2,Geo Corrections Holdings,[Geo Corrections Holdings],[GEO CORRECTIONS HOLDINGS INC],[Geo Corrections Holdings],[GEO CORRECTIONS HOLDINGS INC],"10,000.00","100,000.00",0.09,0.91,"-90,000.00",-0.90
3,Hillwood Development,[Hillwood Development],[HILLWOOD DEVELOPMENT COMPANY LLC],[Hillwood Development],[HILLWOOD DEVELOPMENT COMPANY LLC],"50,000.00","50,000.00",0.50,0.50,0.00,0.00
4,Republican Governors Assn,[Republican Governors Assn],[REPUBLICAN GOVERNORS ASSOCIATION],[Republican Governors Assn],[REPUBLICAN GOVERNORS ASSOCIATION],"750,000.00","3,450,000.00",0.18,0.82,"-2,700,000.00",-0.78


## And did their giving patterns change between the cycles?

In [19]:
donors_15_17 = donors_17.merge(donors_15, how="outer", on="match_id", suffixes=["_17", "_15"])
donors_15_17.drop(["change_17", "pct_change_17", "organizations_post_bill", "contributors_post_bill",
             "organizations_pre_bill", "contributors_pre_bill", "change_15", "pct_change_15"],
            axis=1, inplace=True)
donors_15_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"})
donors_15_17 = donors_15_17[["match_id", "contributors", "organizations", "total_pre_bill_15", "total_post_bill_15",
                 "pct_pre_bill_15", "pct_post_bill_15", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_15_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 11 columns):
match_id              158 non-null object
contributors          158 non-null object
organizations         158 non-null object
total_pre_bill_15     111 non-null float64
total_post_bill_15    111 non-null float64
pct_pre_bill_15       111 non-null float64
pct_post_bill_15      111 non-null float64
total_pre_bill_17     158 non-null float64
total_post_bill_17    158 non-null float64
pct_pre_bill_17       158 non-null float64
pct_post_bill_17      158 non-null float64
dtypes: float64(8), object(3)
memory usage: 14.8+ KB


In [20]:
donors_15_17["giving_change"] = np.where((donors_15_17["pct_post_bill_17"] > donors_15_17["pct_post_bill_15"]) | (donors_15_17["pct_pre_bill_15"].isnull()), "Increased",
                                  np.where(donors_15_17["pct_post_bill_17"] < donors_15_17["pct_post_bill_15"], "Decreased",
                                           np.where(donors_15_17["pct_post_bill_17"] == donors_15_17["pct_post_bill_15"], "Stayed the same",
                                                   "Other")))
donors_15_17.head(1)

,match_id,contributors,organizations,total_pre_bill_15,total_post_bill_15,pct_pre_bill_15,pct_post_bill_15,total_pre_bill_17,total_post_bill_17,pct_pre_bill_17,pct_post_bill_17,giving_change
0,Air Line Pilots Assn,[AIR LINE PILOTS ASSOC INT'L POLITICAL ACTIO...,[Air Line Pilots Assn],nan,nan,nan,nan,"100,000.00","150,000.00",0.40,0.60,Increased


## What proportion of donors increased the share of their annual giving that fell in the last two months of 2017 as compared with the same period in 2015?

In [21]:
donors_15_17[donors_15_17["giving_change"] == "Increased"].count() / donors_15_17.count()

match_id             0.89
contributors         0.89
organizations        0.89
total_pre_bill_15    0.85
total_post_bill_15   0.85
pct_pre_bill_15      0.85
pct_post_bill_15     0.85
total_pre_bill_17    0.89
total_post_bill_17   0.89
pct_pre_bill_17      0.89
pct_post_bill_17     0.89
giving_change        0.89
dtype: float64

## And by how much did these individuals' giving increase?

In [22]:
donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()

28922893.0

In [23]:
donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_15"].sum()

-6589136.0

In [24]:
donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_15_17[donors_15_17["giving_change"] == "Increased"]["total_post_bill_15"].sum()

35512029.0

# What does this giving look like on a contribution-by-contribution level?

During the period after the tax bill was introduced?

In [25]:
post_bill_contributions_17 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_contributions_17 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2017-11-02'
     AND date <= '2017-12-31'
     AND CYCLE = '2018'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2018'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE post_bill_contributions_17 TO redash_default;


SELECT *
FROM post_bill_contributions_17;""", con=conn)
post_bill_contributions_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 8 columns):
fectransid      920 non-null object
match_id        920 non-null object
organization    907 non-null object
contributor     919 non-null object
date            920 non-null object
sum             920 non-null float64
cmte_id         920 non-null object
committee       920 non-null object
dtypes: float64(1), object(7)
memory usage: 57.6+ KB


During the period before the tax bill was introduced?

In [26]:
pre_bill_contributions_17 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_contributions_17 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2017-01-01'
     AND date < '2017-11-02'
     AND CYCLE = '2018'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2018'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE pre_bill_contributions_17 TO redash_default;


SELECT *
FROM pre_bill_contributions_17;""", con=conn)
pre_bill_contributions_17.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1690 entries, 0 to 1689
Data columns (total 8 columns):
fectransid      1690 non-null object
match_id        1690 non-null object
organization    1660 non-null object
contributor     1690 non-null object
date            1690 non-null object
sum             1690 non-null float64
cmte_id         1690 non-null object
committee       1690 non-null object
dtypes: float64(1), object(7)
memory usage: 105.7+ KB


What about during the equivalent post-bill introduction period in 2015?

In [27]:
post_bill_contributions_15 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_contributions_15 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2015-11-02'
     AND date <= '2015-12-31'
     AND CYCLE = '2016'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2016'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE post_bill_contributions_15 TO redash_default;


SELECT *
FROM post_bill_contributions_15;""", con=conn)
post_bill_contributions_15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 615 entries, 0 to 614
Data columns (total 8 columns):
fectransid      615 non-null object
match_id        615 non-null object
organization    606 non-null object
contributor     615 non-null object
date            615 non-null object
sum             615 non-null float64
cmte_id         615 non-null object
committee       615 non-null object
dtypes: float64(1), object(7)
memory usage: 38.5+ KB


In [28]:
pre_bill_contributions_15 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_contributions_15 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2015-01-01'
     AND date < '2015-11-02'
     AND CYCLE = '2016'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2016'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE pre_bill_contributions_15 TO redash_default;


SELECT *
FROM pre_bill_contributions_15;""", con=conn)
pre_bill_contributions_15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2270 entries, 0 to 2269
Data columns (total 8 columns):
fectransid      2270 non-null object
match_id        2270 non-null object
organization    2175 non-null object
contributor     2270 non-null object
date            2270 non-null object
sum             2270 non-null float64
cmte_id         2270 non-null object
committee       2270 non-null object
dtypes: float64(1), object(7)
memory usage: 142.0+ KB


## Had these megadonors given between Nov. 2 and Dec. 31, 2013?

Return contributions by donor and committee.

In [29]:
post_bill_donors_committees_13 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_donors_committees_13 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2013-11-02'
     AND date <= '2013-12-31'
     AND CYCLE = '2014'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2014'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE post_bill_donors_committees_13 TO redash_default;


SELECT *
FROM post_bill_donors_committees_13;""", con=conn)
post_bill_donors_committees_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220 entries, 0 to 219
Data columns (total 6 columns):
match_id         220 non-null object
organizations    220 non-null object
contributors     220 non-null object
total            220 non-null float64
cmte_id          220 non-null object
committee        220 non-null object
dtypes: float64(1), object(5)
memory usage: 10.4+ KB


Group by donor.

In [30]:
post_bill_donors_13 = post_bill_donors_committees_13.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
post_bill_donors_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 4 columns):
match_id         62 non-null object
organizations    62 non-null object
contributors     62 non-null object
total            62 non-null float64
dtypes: float64(1), object(3)
memory usage: 2.0+ KB


## How much did they give?

In [31]:
post_bill_donors_13["total"].sum()

3859464.0

## Had these megadonors given previously in 2013?

Return contributions by donor and committee.

In [32]:
pre_bill_donors_committees_13 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_donors_committees_13 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2013-01-01'
     AND date < '2013-11-02'
     AND CYCLE = '2014'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2014'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE pre_bill_donors_committees_13 TO redash_default;


SELECT *
FROM pre_bill_donors_committees_13;""", con=conn)
pre_bill_donors_committees_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566 entries, 0 to 565
Data columns (total 6 columns):
match_id         566 non-null object
organizations    566 non-null object
contributors     566 non-null object
total            566 non-null float64
cmte_id          566 non-null object
committee        566 non-null object
dtypes: float64(1), object(5)
memory usage: 26.6+ KB


Group by donor.

In [33]:
pre_bill_donors_13 = pre_bill_donors_committees_13.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105 entries, 0 to 104
Data columns (total 4 columns):
match_id         105 non-null object
organizations    105 non-null object
contributors     105 non-null object
total            105 non-null float64
dtypes: float64(1), object(3)
memory usage: 3.4+ KB


## How much did they give?

In [34]:
pre_bill_donors_13["total"].sum()

6716175.0

## How much did these donors give in each period?

In [35]:
donors_13 = post_bill_donors_13.merge(pre_bill_donors_13, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_13["total_pre_bill"].fillna(0, inplace=True)
donors_13["total_post_bill"].fillna(0, inplace=True)
donors_13.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109 entries, 0 to 108
Data columns (total 7 columns):
match_id                   109 non-null object
organizations_post_bill    62 non-null object
contributors_post_bill     62 non-null object
total_post_bill            109 non-null float64
organizations_pre_bill     105 non-null object
contributors_pre_bill      105 non-null object
total_pre_bill             109 non-null float64
dtypes: float64(2), object(5)
memory usage: 6.8+ KB


In [36]:
donors_13["pct_post_bill"] = donors_13["total_post_bill"] / (donors_13["total_pre_bill"] + donors_13["total_post_bill"])
donors_13["pct_pre_bill"] = donors_13["total_pre_bill"] / (donors_13["total_pre_bill"] + donors_13["total_post_bill"])
donors_13["change"] = donors_13["total_post_bill"] - donors_13["total_pre_bill"]
donors_13["pct_change"] = (donors_13["total_post_bill"] - donors_13["total_pre_bill"]) / donors_13["total_pre_bill"].abs()
donors_13 = donors_13[["match_id", "organizations_post_bill", "contributors_post_bill",
                       "organizations_pre_bill", "contributors_pre_bill", "total_post_bill",
                       "total_pre_bill", "pct_post_bill", "pct_pre_bill", "change", "pct_change"]]
donors_13.head()

,match_id,organizations_post_bill,contributors_post_bill,organizations_pre_bill,contributors_pre_bill,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
0,American Action Network,[American Action Network],[AMERICAN ACTION NETWORK],[American Action Network],[AMERICAN ACTION NETWORK],"109,714.00","81,237.00",0.57,0.43,"28,477.00",0.35
1,Anschutz Corp,[Anschutz Corp],[THE ANSCHUTZ CORPORATION],NaN,NaN,"5,000.00",0.00,1.00,0.00,"5,000.00",inf
2,Chickasaw Nation,[Chickasaw Nation],"[CHICKASAW NATION, CHICKASAW NATION, THE, NATI...",[Chickasaw Nation],"[CHICKASAW NATION, CHICKASAW NATION, THE, NATI...","42,800.00","196,500.00",0.18,0.82,"-153,700.00",-0.78
3,Republican Governors Assn,[Republican Governors Assn],[REPUBLICAN GOVERNORS ASSOCIATION],NaN,NaN,"1,000,000.00",0.00,1.00,0.00,"1,000,000.00",inf
4,Shakopee Mdewakanton Sioux Community,[Shakopee Mdewakanton Sioux Community],"[MDEWAKANTON SIOUX COMMUNITY, SHAKOPEE,,,, SHA...",[Shakopee Mdewakanton Sioux Community],"[MDEWAKANTON SIOUX COMMUNITY, SHAKOPEE,,,, SHA...","68,900.00","71,100.00",0.49,0.51,"-2,200.00",-0.03


## And did their giving patterns change between the cycles?

In [37]:
donors_13_17 = donors_17.merge(donors_13, how="outer", on="match_id", suffixes=["_17", "_13"])
donors_13_17.drop(["change_17", "pct_change_17", "organizations_post_bill", "contributors_post_bill",
             "organizations_pre_bill", "contributors_pre_bill", "change_13", "pct_change_13"],
            axis=1, inplace=True)
donors_13_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"})
donors_13_17 = donors_13_17[["match_id", "contributors", "organizations", "total_pre_bill_13", "total_post_bill_13",
                 "pct_pre_bill_13", "pct_post_bill_13", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_13_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 11 columns):
match_id              158 non-null object
contributors          158 non-null object
organizations         158 non-null object
total_pre_bill_13     109 non-null float64
total_post_bill_13    109 non-null float64
pct_pre_bill_13       109 non-null float64
pct_post_bill_13      109 non-null float64
total_pre_bill_17     158 non-null float64
total_post_bill_17    158 non-null float64
pct_pre_bill_17       158 non-null float64
pct_post_bill_17      158 non-null float64
dtypes: float64(8), object(3)
memory usage: 14.8+ KB


In [38]:
donors_13_17["giving_change"] = np.where((donors_13_17["pct_post_bill_17"] > donors_13_17["pct_post_bill_13"]) | (donors_13_17["pct_pre_bill_13"].isnull()), "Increased",
                                  np.where(donors_13_17["pct_post_bill_17"] < donors_13_17["pct_post_bill_13"], "Decreased",
                                           np.where(donors_13_17["pct_post_bill_17"] == donors_13_17["pct_post_bill_13"], "Stayed the same",
                                                   "Other")))
donors_13_17.head(1)

,match_id,contributors,organizations,total_pre_bill_13,total_post_bill_13,pct_pre_bill_13,pct_post_bill_13,total_pre_bill_17,total_post_bill_17,pct_pre_bill_17,pct_post_bill_17,giving_change
0,Air Line Pilots Assn,[AIR LINE PILOTS ASSOC INT'L POLITICAL ACTIO...,[Air Line Pilots Assn],nan,nan,nan,nan,"100,000.00","150,000.00",0.40,0.60,Increased


## What proportion of donors increased the share of their annual giving that fell in the last two months of 2017 as compared with the same period in 2013?

In [39]:
donors_13_17[donors_13_17["giving_change"] == "Increased"].count() / donors_13_17.count()

match_id             0.83
contributors         0.83
organizations        0.83
total_pre_bill_13    0.75
total_post_bill_13   0.75
pct_pre_bill_13      0.75
pct_post_bill_13     0.75
total_pre_bill_17    0.83
total_post_bill_17   0.83
pct_pre_bill_17      0.83
pct_post_bill_17     0.83
giving_change        0.83
dtype: float64

## And by how much did these individuals' giving increase?

In [40]:
donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()

22986493.0

In [41]:
donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_13"].sum()

1155975.0

In [42]:
donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_13_17[donors_13_17["giving_change"] == "Increased"]["total_post_bill_13"].sum()

21830518.0

# What does this giving look like on a contribution-by-contribution level during the equivalent post-bill introduction period in 2013?

In [43]:
post_bill_contributions_13 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_contributions_13 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2013-11-02'
     AND date <= '2013-12-31'
     AND CYCLE = '2014'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2014'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE post_bill_contributions_13 TO redash_default;


SELECT *
FROM post_bill_contributions_13;""", con=conn)
post_bill_contributions_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 335 entries, 0 to 334
Data columns (total 8 columns):
fectransid      335 non-null object
match_id        335 non-null object
organization    326 non-null object
contributor     335 non-null object
date            335 non-null object
sum             335 non-null float64
cmte_id         335 non-null object
committee       335 non-null object
dtypes: float64(1), object(7)
memory usage: 21.0+ KB


In [44]:
pre_bill_contributions_13 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_contributions_13 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2013-01-01'
     AND date < '2013-11-02'
     AND CYCLE = '2014'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2014'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE pre_bill_contributions_13 TO redash_default;


SELECT *
FROM pre_bill_contributions_13;""", con=conn)
pre_bill_contributions_13.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 945 entries, 0 to 944
Data columns (total 8 columns):
fectransid      945 non-null object
match_id        945 non-null object
organization    906 non-null object
contributor     945 non-null object
date            945 non-null object
sum             945 non-null float64
cmte_id         945 non-null object
committee       945 non-null object
dtypes: float64(1), object(7)
memory usage: 59.1+ KB


## Had these megadonors given between Nov. 2 and Dec. 31, 2011?

Return contributions by donor and committee.

In [45]:
post_bill_donors_committees_11 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_donors_committees_11 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2011-11-02'
     AND date <= '2011-12-31'
     AND CYCLE = '2012'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2012'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE post_bill_donors_committees_11 TO redash_default;


SELECT *
FROM post_bill_donors_committees_11;""", con=conn)
post_bill_donors_committees_11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169 entries, 0 to 168
Data columns (total 6 columns):
match_id         169 non-null object
organizations    169 non-null object
contributors     169 non-null object
total            169 non-null float64
cmte_id          169 non-null object
committee        169 non-null object
dtypes: float64(1), object(5)
memory usage: 8.0+ KB


Group by donor.

In [46]:
post_bill_donors_11 = post_bill_donors_committees_11.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
post_bill_donors_11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 4 columns):
match_id         71 non-null object
organizations    71 non-null object
contributors     71 non-null object
total            71 non-null float64
dtypes: float64(1), object(3)
memory usage: 2.3+ KB


## How much did they give?

In [47]:
post_bill_donors_11["total"].sum()

3241839.0

## Had these megadonors given previously in 2011?

Return contributions by donor and committee.

In [48]:
pre_bill_donors_committees_11 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_donors_committees_11 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2011-01-01'
     AND date < '2011-11-02'
     AND CYCLE = '2012'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2012'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE pre_bill_donors_committees_11 TO redash_default;


SELECT *
FROM pre_bill_donors_committees_11;""", con=conn)
pre_bill_donors_committees_11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 6 columns):
match_id         611 non-null object
organizations    611 non-null object
contributors     611 non-null object
total            611 non-null float64
cmte_id          611 non-null object
committee        611 non-null object
dtypes: float64(1), object(5)
memory usage: 28.7+ KB


Group by donor.

In [49]:
pre_bill_donors_11 = pre_bill_donors_committees_11.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 4 columns):
match_id         103 non-null object
organizations    103 non-null object
contributors     103 non-null object
total            103 non-null float64
dtypes: float64(1), object(3)
memory usage: 3.3+ KB


## How much did they give?

In [50]:
pre_bill_donors_11["total"].sum()

7409350.0

## How much did these donors give in each period?

In [51]:
donors_11 = post_bill_donors_11.merge(pre_bill_donors_11, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_11["total_pre_bill"].fillna(0, inplace=True)
donors_11["total_post_bill"].fillna(0, inplace=True)
donors_11.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106 entries, 0 to 105
Data columns (total 7 columns):
match_id                   106 non-null object
organizations_post_bill    71 non-null object
contributors_post_bill     71 non-null object
total_post_bill            106 non-null float64
organizations_pre_bill     103 non-null object
contributors_pre_bill      103 non-null object
total_pre_bill             106 non-null float64
dtypes: float64(2), object(5)
memory usage: 6.6+ KB


In [52]:
donors_11["pct_post_bill"] = donors_11["total_post_bill"] / (donors_11["total_pre_bill"] + donors_11["total_post_bill"])
donors_11["pct_pre_bill"] = donors_11["total_pre_bill"] / (donors_11["total_pre_bill"] + donors_11["total_post_bill"])
donors_11["change"] = donors_11["total_post_bill"] - donors_11["total_pre_bill"]
donors_11["pct_change"] = (donors_11["total_post_bill"] - donors_11["total_pre_bill"]) / donors_11["total_pre_bill"].abs()
donors_11 = donors_11[["match_id", "organizations_post_bill", "contributors_post_bill",
                       "organizations_pre_bill", "contributors_pre_bill", "total_post_bill",
                       "total_pre_bill", "pct_post_bill", "pct_pre_bill", "change", "pct_change"]]
donors_11.head()

,match_id,organizations_post_bill,contributors_post_bill,organizations_pre_bill,contributors_pre_bill,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
0,American Action Network,[American Action Network],[AMERICAN ACTION NETWORK],[American Action Network],[AMERICAN ACTION NETWORK],"7,131.00","4,972.00",0.59,0.41,"2,159.00",0.43
1,Chickasaw Nation,[Chickasaw Nation],"[CHICKASAW NATION, CHICKASAW NATION, THE, NATI...",[Chickasaw Nation],"[CHICKASAW NATION, CHICKASAW NATION, THE, NATI...","23,750.00","270,250.00",0.08,0.92,"-246,500.00",-0.91
2,Republican Governors Assn,[Republican Governors Assn],[REPUBLICAN GOVERNORS ASSOCIATION],[Republican Governors Assn],[REPUBLICAN GOVERNORS ASSOCIATION],"95,000.00","73,605.00",0.56,0.44,"21,395.00",0.29
3,U0000000074,[Hendricks Holding Co],"[HENDRICKS, DIANE M, HENDRICKS, DIANE MS]",[Hendricks Holding Co],"[HENDRICKS, DIANE M, HENDRICKS, DIANE MS]","35,800.00","49,300.00",0.42,0.58,"-13,500.00",-0.27
4,U0000000175,[Stephens Inc],"[STEPHENS, WARREN, STEPHENS, WARREN A MR]",[Stephens Inc],"[STEPHENS, WARREN, STEPHENS, WARREN A MR]","12,500.00","50,000.00",0.20,0.80,"-37,500.00",-0.75


## And did their giving patterns change between the cycles?

In [53]:
donors_11_17 = donors_17.merge(donors_11, how="outer", on="match_id", suffixes=["_17", "_11"])
donors_11_17.drop(["change_17", "pct_change_17", "organizations_post_bill", "contributors_post_bill",
             "organizations_pre_bill", "contributors_pre_bill", "change_11", "pct_change_11"],
            axis=1, inplace=True)
donors_11_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"})
donors_11_17 = donors_11_17[["match_id", "contributors", "organizations", "total_pre_bill_11", "total_post_bill_11",
                 "pct_pre_bill_11", "pct_post_bill_11", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_11_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 11 columns):
match_id              158 non-null object
contributors          158 non-null object
organizations         158 non-null object
total_pre_bill_11     106 non-null float64
total_post_bill_11    106 non-null float64
pct_pre_bill_11       106 non-null float64
pct_post_bill_11      106 non-null float64
total_pre_bill_17     158 non-null float64
total_post_bill_17    158 non-null float64
pct_pre_bill_17       158 non-null float64
pct_post_bill_17      158 non-null float64
dtypes: float64(8), object(3)
memory usage: 14.8+ KB


In [54]:
donors_11_17["giving_change"] = np.where((donors_11_17["pct_post_bill_17"] > donors_11_17["pct_post_bill_11"]) | (donors_11_17["pct_pre_bill_11"].isnull()), "Increased",
                                  np.where(donors_11_17["pct_post_bill_17"] < donors_11_17["pct_post_bill_11"], "Decreased",
                                           np.where(donors_11_17["pct_post_bill_17"] == donors_11_17["pct_post_bill_11"], "Stayed the same",
                                                   "Other")))
donors_11_17.head(1)

,match_id,contributors,organizations,total_pre_bill_11,total_post_bill_11,pct_pre_bill_11,pct_post_bill_11,total_pre_bill_17,total_post_bill_17,pct_pre_bill_17,pct_post_bill_17,giving_change
0,Air Line Pilots Assn,[AIR LINE PILOTS ASSOC INT'L POLITICAL ACTIO...,[Air Line Pilots Assn],"2,500.00",0.00,1.00,0.00,"100,000.00","150,000.00",0.40,0.60,Increased


## What proportion of donors increased the share of their annual giving that fell in the last two months of 2017 as compared with the same period in 2011?

In [55]:
donors_11_17[donors_11_17["giving_change"] == "Increased"].count() / donors_11_17.count()

match_id             0.85
contributors         0.85
organizations        0.85
total_pre_bill_11    0.77
total_post_bill_11   0.77
pct_pre_bill_11      0.77
pct_post_bill_11     0.77
total_pre_bill_17    0.85
total_post_bill_17   0.85
pct_pre_bill_17      0.85
pct_post_bill_17     0.85
giving_change        0.85
dtype: float64

## And by how much did these individuals' giving increase?

In [56]:
donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()

24904443.0

In [57]:
donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_11"].sum()

613592.0

In [58]:
donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_11_17[donors_11_17["giving_change"] == "Increased"]["total_post_bill_11"].sum()

24290851.0

# What does this giving look like on a contribution-by-contribution level during the equivalent post-bill introduction period in 2011?

In [59]:
post_bill_contributions_11 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_contributions_11 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2011-11-02'
     AND date <= '2011-12-31'
     AND CYCLE = '2012'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2012'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE post_bill_contributions_11 TO redash_default;


SELECT *
FROM post_bill_contributions_11;""", con=conn)
post_bill_contributions_11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 270 entries, 0 to 269
Data columns (total 8 columns):
fectransid      270 non-null object
match_id        270 non-null object
organization    268 non-null object
contributor     270 non-null object
date            270 non-null object
sum             270 non-null float64
cmte_id         270 non-null object
committee       270 non-null object
dtypes: float64(1), object(7)
memory usage: 17.0+ KB


In [60]:
pre_bill_contributions_11 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_contributions_11 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2011-01-01'
     AND date < '2011-11-02'
     AND CYCLE = '2012'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2012'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE pre_bill_contributions_11 TO redash_default;


SELECT *
FROM pre_bill_contributions_11;""", con=conn)
pre_bill_contributions_11.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1047 entries, 0 to 1046
Data columns (total 8 columns):
fectransid      1047 non-null object
match_id        1047 non-null object
organization    1036 non-null object
contributor     1047 non-null object
date            1047 non-null object
sum             1047 non-null float64
cmte_id         1047 non-null object
committee       1047 non-null object
dtypes: float64(1), object(7)
memory usage: 65.5+ KB


## Had these megadonors given between Nov. 2 and Dec. 31, 2009?

Return contributions by donor and committee.

In [61]:
post_bill_donors_committees_09 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_donors_committees_09 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2009-11-02'
     AND date <= '2009-12-31'
     AND CYCLE = '2010'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2010'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE post_bill_donors_committees_09 TO redash_default;


SELECT *
FROM post_bill_donors_committees_09;""", con=conn)
post_bill_donors_committees_09.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 6 columns):
match_id         120 non-null object
organizations    120 non-null object
contributors     120 non-null object
total            120 non-null float64
cmte_id          120 non-null object
committee        120 non-null object
dtypes: float64(1), object(5)
memory usage: 5.7+ KB


Group by donor.

In [62]:
post_bill_donors_09 = post_bill_donors_committees_09.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
post_bill_donors_09.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
match_id         50 non-null object
organizations    50 non-null object
contributors     50 non-null object
total            50 non-null float64
dtypes: float64(1), object(3)
memory usage: 1.6+ KB


## How much did they give?

In [63]:
post_bill_donors_09["total"].sum()

671846.0

## Had these megadonors given previously in 2009?

Return contributions by donor and committee.

In [64]:
pre_bill_donors_committees_09 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_donors_committees_09 AS
SELECT match_id,
       organizations,
       contributors,
       sum(committee_total) AS total,
       cmte_id,
       pacshort AS committee
FROM
  (SELECT match_id,
          organizations,
          contributors,
          sum(amount) AS committee_total,
          cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id,
             array_agg(DISTINCT orgname) AS organizations,
             array_agg(DISTINCT contrib) AS contributors
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2009-01-01'
     AND date < '2009-11-02'
     AND CYCLE = '2010'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY match_id,
            organizations,
            contributors,
            cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2010'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY match_id,
         organizations,
         contributors,
         cmte_id,
         committee;

GRANT ALL ON TABLE pre_bill_donors_committees_09 TO redash_default;


SELECT *
FROM pre_bill_donors_committees_09;""", con=conn)
pre_bill_donors_committees_09.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 325 entries, 0 to 324
Data columns (total 6 columns):
match_id         325 non-null object
organizations    325 non-null object
contributors     325 non-null object
total            325 non-null float64
cmte_id          325 non-null object
committee        325 non-null object
dtypes: float64(1), object(5)
memory usage: 15.3+ KB


Group by donor.

In [65]:
pre_bill_donors_09 = pre_bill_donors_committees_09.groupby(["match_id"]).agg({"organizations": max, "contributors": max, "total": sum}).reset_index()
pre_bill_donors_09.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 81 entries, 0 to 80
Data columns (total 4 columns):
match_id         81 non-null object
organizations    81 non-null object
contributors     81 non-null object
total            81 non-null float64
dtypes: float64(1), object(3)
memory usage: 2.6+ KB


## How much did they give?

In [66]:
pre_bill_donors_09["total"].sum()

1949651.0

## How much did these donors give in each period?

In [67]:
donors_09 = post_bill_donors_09.merge(pre_bill_donors_09, how="outer", on="match_id", suffixes=["_post_bill", "_pre_bill"])
donors_09["total_pre_bill"].fillna(0, inplace=True)
donors_09["total_post_bill"].fillna(0, inplace=True)
donors_09.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88 entries, 0 to 87
Data columns (total 7 columns):
match_id                   88 non-null object
organizations_post_bill    50 non-null object
contributors_post_bill     50 non-null object
total_post_bill            88 non-null float64
organizations_pre_bill     81 non-null object
contributors_pre_bill      81 non-null object
total_pre_bill             88 non-null float64
dtypes: float64(2), object(5)
memory usage: 5.5+ KB


In [68]:
donors_09["pct_post_bill"] = donors_09["total_post_bill"] / (donors_09["total_pre_bill"] + donors_09["total_post_bill"])
donors_09["pct_pre_bill"] = donors_09["total_pre_bill"] / (donors_09["total_pre_bill"] + donors_09["total_post_bill"])
donors_09["change"] = donors_09["total_post_bill"] - donors_09["total_pre_bill"]
donors_09["pct_change"] = (donors_09["total_post_bill"] - donors_09["total_pre_bill"]) / donors_09["total_pre_bill"].abs()
donors_09 = donors_09[["match_id", "organizations_post_bill", "contributors_post_bill",
                       "organizations_pre_bill", "contributors_pre_bill", "total_post_bill",
                       "total_pre_bill", "pct_post_bill", "pct_pre_bill", "change", "pct_change"]]
donors_09.head()

,match_id,organizations_post_bill,contributors_post_bill,organizations_pre_bill,contributors_pre_bill,total_post_bill,total_pre_bill,pct_post_bill,pct_pre_bill,change,pct_change
0,Chickasaw Nation,[Chickasaw Nation],"[CHICKASAW NATION, CHICKASAW NATION, THE, NATI...",[Chickasaw Nation],"[CHICKASAW NATION, CHICKASAW NATION, THE, NATI...","75,600.00","96,700.00",0.44,0.56,"-21,100.00",-0.22
1,U0000000074,[Hendricks Holding Co],"[HENDRICKS, DIANE M, HENDRICKS, DIANE MS]",[Hendricks Holding Co],"[HENDRICKS, DIANE M, HENDRICKS, DIANE MS]","10,500.00","34,150.00",0.24,0.76,"-23,650.00",-0.69
2,U0000000175,[Stephens Inc],"[STEPHENS, WARREN, STEPHENS, WARREN A MR]",[Stephens Inc],"[STEPHENS, WARREN, STEPHENS, WARREN A MR]","4,800.00","43,700.00",0.10,0.90,"-38,900.00",-0.89
3,U0000000283,"[Broidy Capital Management, Colfax Law Office]","[BROIDY, ELLIOTT, BROIDY, ELLIOTT B MR, BROIDY...","[Broidy Capital Management, Colfax Law Office]","[BROIDY, ELLIOTT, BROIDY, ELLIOTT B MR, BROIDY...",0.00,"91,089.00",0.00,1.00,"-91,089.00",-1.00
4,U0000000320,[TD Ameritrade],"[RICKETTS, MARLENE, RICKETTS, MARLENE M, RICKE...",[TD Ameritrade],"[RICKETTS, MARLENE, RICKETTS, MARLENE M, RICKE...","11,600.00","4,000.00",0.74,0.26,"7,600.00",1.90


## And did their giving patterns change between the cycles?

In [69]:
donors_09_17 = donors_17.merge(donors_09, how="outer", on="match_id", suffixes=["_17", "_09"])
donors_09_17.drop(["change_17", "pct_change_17", "organizations_post_bill", "contributors_post_bill",
             "organizations_pre_bill", "contributors_pre_bill", "change_09", "pct_change_09"],
            axis=1, inplace=True)
donors_09_17.rename(columns={"organizations_17": "organizations", "contributors_17": "contributors"})
donors_09_17 = donors_09_17[["match_id", "contributors", "organizations", "total_pre_bill_09", "total_post_bill_09",
                 "pct_pre_bill_09", "pct_post_bill_09", "total_pre_bill_17", "total_post_bill_17",
                 "pct_pre_bill_17", "pct_post_bill_17"]]
donors_09_17.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 0 to 157
Data columns (total 11 columns):
match_id              158 non-null object
contributors          158 non-null object
organizations         158 non-null object
total_pre_bill_09     88 non-null float64
total_post_bill_09    88 non-null float64
pct_pre_bill_09       88 non-null float64
pct_post_bill_09      88 non-null float64
total_pre_bill_17     158 non-null float64
total_post_bill_17    158 non-null float64
pct_pre_bill_17       158 non-null float64
pct_post_bill_17      158 non-null float64
dtypes: float64(8), object(3)
memory usage: 14.8+ KB


In [70]:
donors_09_17["giving_change"] = np.where((donors_09_17["pct_post_bill_17"] > donors_09_17["pct_post_bill_09"]) | (donors_09_17["pct_pre_bill_09"].isnull()), "Increased",
                                  np.where(donors_09_17["pct_post_bill_17"] < donors_09_17["pct_post_bill_09"], "Decreased",
                                           np.where(donors_09_17["pct_post_bill_17"] == donors_09_17["pct_post_bill_09"], "Stayed the same",
                                                   "Other")))
donors_09_17.head(1)

,match_id,contributors,organizations,total_pre_bill_09,total_post_bill_09,pct_pre_bill_09,pct_post_bill_09,total_pre_bill_17,total_post_bill_17,pct_pre_bill_17,pct_post_bill_17,giving_change
0,Air Line Pilots Assn,[AIR LINE PILOTS ASSOC INT'L POLITICAL ACTIO...,[Air Line Pilots Assn],nan,nan,nan,nan,"100,000.00","150,000.00",0.40,0.60,Increased


## What proportion of donors increased the share of their annual giving that fell in the last two months of 2017 as compared with the same period in 2009?

In [71]:
donors_09_17[donors_09_17["giving_change"] == "Increased"].count() / donors_09_17.count()

match_id             0.85
contributors         0.85
organizations        0.85
total_pre_bill_09    0.74
total_post_bill_09   0.74
pct_pre_bill_09      0.74
pct_post_bill_09     0.74
total_pre_bill_17    0.85
total_post_bill_17   0.85
pct_pre_bill_17      0.85
pct_post_bill_17     0.85
giving_change        0.85
dtype: float64

## And by how much did these individuals' giving increase?

In [72]:
donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_17"].sum()

32149263.0

In [73]:
donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_09"].sum()

267350.0

In [74]:
donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_17"].sum() - donors_09_17[donors_09_17["giving_change"] == "Increased"]["total_post_bill_09"].sum()

31881913.0

# What does this giving look like on a contribution-by-contribution level during the equivalent post-bill introduction period in 2009?

In [75]:
post_bill_contributions_09 = pd.read_sql("""CREATE TABLE IF NOT EXISTS post_bill_contributions_09 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2009-11-02'
     AND date <= '2009-12-31'
     AND CYCLE = '2010'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2010'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE post_bill_contributions_09 TO redash_default;


SELECT *
FROM post_bill_contributions_09;""", con=conn)
post_bill_contributions_09.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187 entries, 0 to 186
Data columns (total 8 columns):
fectransid      187 non-null object
match_id        187 non-null object
organization    184 non-null object
contributor     187 non-null object
date            187 non-null object
sum             187 non-null float64
cmte_id         187 non-null object
committee       187 non-null object
dtypes: float64(1), object(7)
memory usage: 11.8+ KB


In [76]:
pre_bill_contributions_09 = pd.read_sql("""CREATE TABLE IF NOT EXISTS pre_bill_contributions_09 AS
SELECT fectransid,
       match_id,
       organization,
       contributor, date, sum(SUM),
                          cmte_id,
                          pacshort AS committee
FROM
  (SELECT fectransid,
          match_id,
          orgname AS organization,
          contrib AS contributor, date, sum(amount),
                                        cmteid AS cmte_id
   FROM
     (SELECT CASE
                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                 ELSE orgname
             END AS match_id
      FROM crp_contributions
      LEFT JOIN crp_committees ON crp_contributions.cmteid = crp_committees.cmteid
      AND crp_committees.cycle = '2018'
      WHERE primcode IN ('J1100',
                         'J2200',
                         'J2400',
                         'Z1100',
                         'Z4100',
                         'Z4500',
                         'Z5100')
        AND date >= '2017-11-02'
        AND date <= '2017-12-31'
        AND crp_contributions.cycle = '2018'
        AND TYPE NOT IN ('11J',
                         '15C',
                         '15E',
                         '15I',
                         '15J',
                         '15T',
                         '18J',
                         '19J',
                         '30J',
                         '30F',
                         '31J',
                         '31F',
                         '32J',
                         '32F')
      GROUP BY match_id
      HAVING sum(amount) >= 50000) AS gop_donors
   JOIN crp_contributions ON CASE
                                 WHEN trim(contribid) != '' THEN left(contribid, 11)
                                 ELSE crp_contributions.orgname
                             END = gop_donors.match_id
   WHERE date >= '2009-01-01'
     AND date < '2009-11-02'
     AND CYCLE = '2010'
     AND TYPE NOT IN ('11J',
                      '15C',
                      '15E',
                      '15I',
                      '15J',
                      '15T',
                      '18J',
                      '19J',
                      '30J',
                      '30F',
                      '31J',
                      '31F',
                      '32J',
                      '32F')
   GROUP BY fectransid,
            match_id,
            organization,
            contributor, date, cmte_id) AS donors_committees
JOIN crp_committees ON donors_committees.cmte_id = crp_committees.cmteid
AND crp_committees.CYCLE = '2010'
WHERE primcode IN ('J1100',
                   'J2200',
                   'J2400',
                   'Z1100',
                   'Z4100',
                   'Z4500',
                   'Z5100')
GROUP BY fectransid,
         match_id,
         organization,
         contributor, date, cmte_id,
                            committee;

GRANT ALL ON TABLE pre_bill_contributions_09 TO redash_default;


SELECT *
FROM pre_bill_contributions_09;""", con=conn)
pre_bill_contributions_09.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 520 entries, 0 to 519
Data columns (total 8 columns):
fectransid      520 non-null object
match_id        520 non-null object
organization    508 non-null object
contributor     520 non-null object
date            520 non-null object
sum             520 non-null float64
cmte_id         520 non-null object
committee       520 non-null object
dtypes: float64(1), object(7)
memory usage: 32.6+ KB


## Export data to Excel.

In [77]:
writer = pd.ExcelWriter("data/megadonors.xlsx")
donors_15_17.to_excel(writer, "donors_15_17", startcol=0, index=False)
donors_13_17.to_excel(writer, "donors_13_17", startcol=0, index=False)
donors_11_17.to_excel(writer, "donors_11_17", startcol=0, index=False)
donors_09_17.to_excel(writer, "donors_09_17", startcol=0, index=False)
post_bill_donors_committees_17.to_excel(writer, "post_bill_donors_committees_17", startcol=0, index=False)
pre_bill_donors_committees_17.to_excel(writer, "pre_bill_donors_committees_17", startcol=0, index=False)
post_bill_donors_committees_15.to_excel(writer, "post_bill_donors_committees_15", startcol=0, index=False)
pre_bill_donors_committees_15.to_excel(writer, "pre_bill_donors_committees_15", startcol=0, index=False)
post_bill_donors_committees_13.to_excel(writer, "post_bill_donors_committees_13", startcol=0, index=False)
pre_bill_donors_committees_13.to_excel(writer, "pre_bill_donors_committees_13", startcol=0, index=False)
post_bill_donors_committees_11.to_excel(writer, "post_bill_donors_committees_11", startcol=0, index=False)
pre_bill_donors_committees_11.to_excel(writer, "pre_bill_donors_committees_11", startcol=0, index=False)
post_bill_donors_committees_09.to_excel(writer, "post_bill_donors_committees_09", startcol=0, index=False)
pre_bill_donors_committees_09.to_excel(writer, "pre_bill_donors_committees_09", startcol=0, index=False)
post_bill_contributions_17.to_excel(writer, "post_bill_contributions_17", startcol=0, index=False)
pre_bill_contributions_17.to_excel(writer, "pre_bill_contributions_17", startcol=0, index=False)
post_bill_contributions_15.to_excel(writer, "post_bill_contributions_15", startcol=0, index=False)
pre_bill_contributions_15.to_excel(writer, "pre_bill_contributions_15", startcol=0, index=False)
post_bill_contributions_13.to_excel(writer, "post_bill_contributions_13", startcol=0, index=False)
pre_bill_contributions_13.to_excel(writer, "pre_bill_contributions_13", startcol=0, index=False)
post_bill_contributions_11.to_excel(writer, "post_bill_contributions_11", startcol=0, index=False)
pre_bill_contributions_11.to_excel(writer, "pre_bill_contributions_11", startcol=0, index=False)
post_bill_contributions_09.to_excel(writer, "post_bill_contributions_09", startcol=0, index=False)
pre_bill_contributions_09.to_excel(writer, "pre_bill_contributions_09", startcol=0, index=False)
writer.save()